In [0]:
import pandas as pd
from matplotlib import pyplot
from sklearn.externals import joblib
import numpy as np
import datetime
import pickle

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path= "./drive/My Drive/uk dale dataset/channel_2.dat"

In [0]:
def string_to_timestamp(string):
    date_time_obj = datetime.datetime.strptime(string, '%Y-%m-%d %H:%M:%S')
    timestamp = date_time_obj.timestamp()
    return timestamp

In [0]:
class data_prep():
    def __init__(self, path):
      self.df = pd.read_csv(path, sep='\s+', header=None)

    def resampler(self,time = "5min"):
      dataframe = self.df.set_index([0])
      dataframe.index = pd.to_datetime(dataframe.index,unit = "s")
      resample = dataframe.resample(time)
      self.resampled_data = resample.sum()

    def get_resampled_with_timestamp(self):
      self.final_data = self.resampled_data.reset_index()

    def get_final_data(self):
      self.resampler()
      self.get_resampled_with_timestamp()
      return self.final_data

In [0]:
prep = data_prep(path)
data = prep.get_final_data()

In [99]:
data.head(5)

,0,1
0,2012-11-09 22:25:00,409
1,2012-11-09 22:30:00,965
2,2012-11-09 22:35:00,967
3,2012-11-09 22:40:00,917
4,2012-11-09 22:45:00,380


In [0]:
class to_serialised_on_off():
    def __init__(self, data):
      self.data = data
      self.min_ = 0
      self.avg_ = 0
      self.max_ = 0
      self.find_min_max(flag = 1)

    def append_to_df(self, val):
      self.data.append(val)
      self.find_min_max()

    def find_min_max(self, flag = 0):
      if(flag == 0):
        val = self.data.loc[-1][-1]
        if( val > self.max_ ):
          self.max_ = val
        elif( val < self.min_ ):
          self.min_ = val
        self.avg_ += val/self.data.shape[0]
      elif(flag):
        self.max_ = max(self.data.loc[:][1])
        self.min_ = min(self.data.loc[:][1])
        self.avg_ = np.mean(self.data.loc[:][1].values)

      self.calc_thresh()

    def calc_thresh(self):
      self.thresh = (self.max_ - self.min_)/self.avg_

    def on_off(self):
      of = []
      for i in range(self.data.shape[0]):
        if(self.data.loc[i][1] > self.thresh):
          of.append(str(self.data.loc[i][0]))
        else:
          of.append(0)

      return of

In [0]:
obj1 = to_serialised_on_off(data[:1000])

In [0]:
arr = obj1.on_off()

In [129]:
print(arr)

['2012-11-09 22:25:00', '2012-11-09 22:30:00', '2012-11-09 22:35:00', '2012-11-09 22:40:00', '2012-11-09 22:45:00', '2012-11-09 22:50:00', '2012-11-09 22:55:00', '2012-11-09 23:00:00', '2012-11-09 23:05:00', '2012-11-09 23:10:00', '2012-11-09 23:15:00', '2012-11-09 23:20:00', '2012-11-09 23:25:00', '2012-11-09 23:30:00', '2012-11-09 23:35:00', '2012-11-09 23:40:00', '2012-11-09 23:45:00', 0, '2012-11-09 23:55:00', '2012-11-10 00:00:00', '2012-11-10 00:05:00', '2012-11-10 00:10:00', '2012-11-10 00:15:00', '2012-11-10 00:20:00', '2012-11-10 00:25:00', '2012-11-10 00:30:00', '2012-11-10 00:35:00', '2012-11-10 00:40:00', '2012-11-10 00:45:00', '2012-11-10 00:50:00', '2012-11-10 00:55:00', '2012-11-10 01:00:00', '2012-11-10 01:05:00', '2012-11-10 01:10:00', '2012-11-10 01:15:00', '2012-11-10 01:20:00', '2012-11-10 01:25:00', '2012-11-10 01:30:00', '2012-11-10 01:35:00', '2012-11-10 01:40:00', '2012-11-10 01:45:00', '2012-11-10 01:50:00', '2012-11-10 01:55:00', '2012-11-10 02:00:00', '2012-1

In [114]:
print(obj1.thresh, obj1.max_, obj1.min_, obj1.avg_)

10.242317724559646 4767 0 465.422
